In [ ]:
#!python -m spacy download en_core_web_trf

In [ ]:
#!pip install spacy-transformers

In [8]:
from pdfminer.high_level import extract_text
import spacy
import spacy_transformers
nlp = spacy.load("en_core_web_trf")
import re
import pandas as pd
from spacy.lang.en.stop_words import STOP_WORDS

In [45]:
text = extract_text('2004-envoy.pdf')
text = re.sub(r'\n|\t|\r',' ',text)
pattern = r'2HI|4HI|A4WD'
text = re.sub(pattern,'four-wheel drive',text)

In [46]:
doc = nlp(text)

In [47]:
# grab all nouns to start. 
entities = []
for sent in doc.sents:
    #cutting out table of contents/index
    if '.......' in sent.text:
        pass
    else:
        phrase = []
        nouns = []
        subjects = []
        objects = []
        nsubjects = []
        dobjects = []
        for chunk in sent.noun_chunks:
            phrase.append(chunk.text)
            #simple filter t get rid of useless nouns
            if len(chunk.root.text)>2:
                t = chunk.root
                if t.pos_ == "NOUN":
                    nouns.append(t.lemma_)
                if 'obj' in t.dep_:
                    objects.append(t.lemma_)
                if 'subj' in t.dep_:
                    subjects.append(t.lemma_)
                if 'dobj' in t.dep_:
                    dobjects.append(t.lemma_)
                if 'nsubj' in t.dep_:
                    nsubjects.append(t.lemma_)
                entities.append([phrase, nouns, objects, subjects,dobjects, nsubjects, sent.text])
pos_df =  pd.DataFrame(entities, columns=['phrase','nouns', 'objects', 'subjects','dobjects','nsubjects','sent'])
pos_df.to_csv('basic_pos.csv', index=False)

In [13]:
pos_df.nouns

0        [seat, seat, belt, restraint, system, system, ...
1        [seat, seat, belt, restraint, system, system, ...
2        [seat, seat, belt, restraint, system, system, ...
3        [seat, seat, belt, restraint, system, system, ...
4        [seat, seat, belt, restraint, system, system, ...
                               ...                        
23805                              [resident, check, fund]
23806                              [resident, check, fund]
23807                                              [grade]
23808                                            [trailer]
23809                                          [satellite]
Name: nouns, Length: 23810, dtype: object

In [ ]:
test =  nlp('Each of these surfaces affects the steering, acceleration and braking of your vehicle in different ways.')
for t in test:
    if t.pos_=='NOUN':
        print(t)

In [ ]:
#what is the connection between power steering and 4wheel drive

In [ ]:
# remember noun chunks when adding the advanced features

# OLD

In [ ]:
#while I like this much better, I have to keep in mind that this needs to be as simple as possible. 
phrase = nlp("'Move the front of the horizontal seat control up or down to adjust the front portion of the cushion'")
relationships = []
for sent in doc.sents:
    root, subj, subj_head, obj, obj_head = '','','','',''
    root = ','.join([t.lemma_ for t in sent if t.dep_=='ROOT'])
    for t in sent:
        if 'subj' in t.dep_:
            subj_head = t.head.lemma_
            subj = t.lemma_
        #find object
        # need to build seperate parser for prepositions.  
        if 'dobj' in t.dep_:
            obj =  t.lemma_
            obj_head = t.head.lemma_
            relationships.append([subj,subj_head, obj, obj_head, root, sent])
rel_df = pd.DataFrame(relationships, columns = ['subj','subj_head', 'obj', 'obj_head', 'root', 'sent'])
rel_df.to_csv('rel_df.csv', index=False)